In [ ]:

def sigmoid(z):
    z=np.array(z)
    return 1/(1+np.exp(-z).astype('float64'))

In [ ]:
def forward_prop(X,Y,b1,W1,W2,b2,reg):
    Z=np.dot(X,W1)+np.transpose(b1)
    A=sigmoid(Z)
    y_hat=np.dot(A,W2)+np.transpose(b2)
    m=X.shape[0]
    mse =1/m* np.sum(np.square((Y - y_hat)**2))
    Loss=mse+ reg*(np.sum(W1**2)+np.sum(W2**2))
    cache={"y_hat":y_hat,"Z":Z,"A":A}
    return Loss,cache

In [ ]:

def back_prop(X,Y,W1,W2,b1,b2,cache,reg):
    m=Y.shape[0]
    y_hat=cache["y_hat"]
    A=cache["A"]
    Z=cache["Z"]
    dy_hat =1/m*2*(y_hat-Y)
    dW2=np.dot(A.T,dy_hat) + reg*W2
    db2 =np.sum(dy_hat,axis=0)
    dA = np.dot(y_hat,W2.T)
    dZ = dA *A*(1-A)
    dW1 =np.dot(X.T, dZ)+reg*W1
    db1=np.sum(dZ,axis=0)
    gradients={"dW1":dW1,"db1":db1,"dW2":dW2,"db2":db2}
    return gradients

In [ ]:
def accuracy(y_hat,y):
    y_hat = np.argmax(y_hat,axis=1)
    y= np.argmax(y,axis=1)
    accuracy=np.sum(y_hat==y)/y.size 
    return accuracy

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
print('x_train:' , x_train.shape)
H=200#no of hidden layers
K = len(np.unique(y_train)) # Classes
Ntr = x_train.shape[0]
Nte = x_test.shape[0]
Din = 3072 # CIFAR10
# Din = 784 # MINIST
# Normalize pixel values
x_train, x_test = x_train/255.0 , x_test/255.0 
mean_image = np.mean(x_train, axis=0)
x_train = x_train - mean_image
x_test = x_test - mean_image

y_train = tf.keras.utils.to_categorical(y_train, num_classes=K)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=K)
x_train = np.reshape(x_train,(Ntr,Din))#shaping images into a vector Din=32x32x3
x_test = np.reshape(x_test,(Nte,Din))
x_train = x_train.astype('float64')
x_test = x_test.astype('float64')
std=1e-4
W1 = std*np.random.randn(Din, H)#W1 initialization
b1 = np.zeros(H)#b1 initialization
W2 = std*np.random.randn(H, K)#W2 initialization
b2 = np.zeros(K)#b2 initialization
print("W1:", W1.shape)
print("b1:", b1.shape)
print("W2:", W2.shape)
print("b2:", b2.shape)
batch_size = Ntr #for the batch size whole dataset is considered

iterations =300
lr =0.012
lr_decay=0.9999#learning rate is decreased in each iteration
reg =1e-5#regularization term which stops growth of weights
train_loss_history = []
test_loss_history=[]
train_acc_history = []
test_acc_history = []

seed = 0
rng = np.random.default_rng(seed=seed)
for t in range(iterations):
    indices = np.arange(Ntr)#generating image
    rng.shuffle(indices)
    x_train=x_train[indices]*255
    y_train=y_train[indices]
    
    # Forward pass
    loss,cache=forward_prop(W1=W1,b1=b1,W2=W2,b2=b2,X=x_train,Y=y_train,reg=reg)
    train_acc=accuracy(cache["y_hat"],y_train)
    train_acc_history.append(train_acc)
    train_loss_history.append(loss)

    test_loss,test_cache=forward_prop(X=x_test,Y=y_test,W1=W1,b1=b1,W2=W2,b2=b2,reg=reg)
    test_acc=accuracy(test_cache["y_hat"],y_test)
    test_acc_history.append(test_acc)
    test_loss_history.append(test_loss)

    if t%10==0:
        print("Epoch "+str(t+1)+"/"+str(iterations)+">>> trainig_loss: ",loss,">>training_accuracy: ",train_acc,">>testing_loss:",test_loss,">>>testing_accuracy:",test_acc)

    # Backward pass
    gradients=back_prop(X=x_train,Y=y_train,W1=W1,W2=W2,b1=b1,b2=b2,cache=cache,reg=reg)
    dW1=gradients["dW1"]
    db1=gradients["db1"]
    dW2=gradients["dW2"]
    db2=gradients["db2"]

    W1-=lr*dW1
    b1-=lr*b1
    W2-=lr*dW2
    b2-=lr*db2
    lr*=lr_decay

